In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# counties_senate_districts = gpd.read_file('data/election/cb_2018_us_county_within_senate_districts/cb_2018_us_county_within_cd116_500k.shp')
counties = gpd.read_file('data/election/cb_2018_us_county_5m/cb_2018_us_county_5m.shp')
election_senate_2022_county = pd.read_csv('data/election/senate_2022.csv')
education_counties = pd.read_csv('data/election/us_counties_education_1970_2022.csv')
unemployment_counties = pd.read_csv('data/election/us_counties_unemployment_2000_2022.csv')
poverity_counties = pd.read_csv('data/election/us_counties_poverty_2021.csv')
population_counties = pd.read_csv('data/election/us_counties_population_2020_2023.csv')

In [ ]:
# for all STATEFP in counties, remove the leading 0
counties['STATEFP'] = counties['STATEFP'].astype(int).astype(str)

In [ ]:
counties['FIPS Code'] = counties['STATEFP'] + counties['COUNTYFP']

In [ ]:
education_counties['FIPS Code'] = education_counties['FIPS Code'].astype(str)
unemployment_counties['FIPS Code'] = unemployment_counties['FIPS_Code'].astype(str)
poverity_counties['FIPS Code'] = poverity_counties['FIPS_Code'].astype(str)
population_counties['FIPS Code'] = population_counties['FIPStxt'].astype(str)

In [ ]:
# merge the education data with the counties data on the FIPS Code column
education_counties_merged = education_counties.merge(counties, on='FIPS Code')
# merge the unemployment data with the counties data on the FIPS Code column
unemployment_counties_merged = unemployment_counties.merge(counties, on='FIPS Code')
# merge the poverity data with the counties data on the FIPS Code column
poverity_counties_merged = poverity_counties.merge(counties, on='FIPS Code')
# merge the population data with the counties data on the FIPS Code column
population_counties_merged = population_counties.merge(counties, on='FIPS Code')

In [ ]:
poverity_counties_merged.drop(columns=['STATEFP', 'COUNTYFP',
       'COUNTYNS', 'AFFGEOID', 'GEOID', 'NAME', 'LSAD', 'ALAND', 'AWATER',
       'geometry'], inplace=True)
unemployment_counties_merged.drop(columns=['STATEFP', 'COUNTYFP',
       'COUNTYNS', 'AFFGEOID', 'GEOID', 'NAME', 'LSAD', 'ALAND', 'AWATER',
       'geometry'], inplace=True)
population_counties_merged.drop(columns=['STATEFP', 'COUNTYFP',
       'COUNTYNS', 'AFFGEOID', 'GEOID', 'NAME', 'LSAD', 'ALAND', 'AWATER',
       'geometry'], inplace=True)

In [ ]:
# merge all the dataframes
merged = education_counties_merged.merge(unemployment_counties_merged, on='FIPS Code')
merged = merged.merge(poverity_counties_merged, on='FIPS Code')
merged = merged.merge(population_counties_merged, on='FIPS Code')

In [ ]:
merged_geo = gpd.GeoDataFrame(merged, geometry=merged['geometry'])

In [ ]:
# drop columns from merged_geo containing any of the following strings in the col name: 'Urban', 'Rural', '1970', '1980', '1990'
cols_to_drop = [col for col in merged_geo.columns if 'Urban' in col or 'Rural' in col or '1970' in col or '1980' in col or '1990' in col]
merged_geo.drop(columns=cols_to_drop, inplace=True)
merged_geo.drop(columns=['STATEFP', 'COUNTYFP', 'COUNTYNS', 'AFFGEOID', 'GEOID', 'NAME', 'LSAD'], inplace=True)
merged_geo.drop(columns=['FIPS_Code_x', 'State_y', 'Area_Name_x', 'FIPS_Code_y', 'Stabr', 'Area_name', 'State', 'Area_Name_y'], inplace=True)
merged_geo.rename(columns={'State_x': 'State'}, inplace=True)
merged_geo = merged_geo[~merged_geo['State'].isin(['AK', 'HI', 'DC'])]
merged_geo.head()

In [ ]:
# in counties, show entries where NAME is 'Orleans'
counties[counties['NAME'] == 'Orleans']

In [ ]:
# save the merged data to a geojson file
merged_geo.to_file('data/election/final_data/counties_data.geojson', driver='GeoJSON')

In [ ]:
# read the geojson file
county_level_data = gpd.read_file('data/election/final_data/counties_data.geojson')

In [ ]:
county_level_data.head()

In [ ]:
# add a column to county level data that is the full name of the state, call it State_name
state_names = {
    'AL': 'Alabama',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming'
}

county_level_data['State_name'] = county_level_data['State'].map(state_names)

In [ ]:
county_level_data.to_file('data/election/final_data/counties_data.geojson', driver='GeoJSON')

In [ ]:
# create a column called state_county that is the concatenation of the state and county name, underscore separated
county_level_data['state_county'] = county_level_data['State_name'] + '_' + county_level_data['Area name']

In [ ]:
# change state_county to lower case
county_level_data['state_county'] = county_level_data['state_county'].str.lower()

In [ ]:
# county_level_data[~county_level_data['state_county'].str.contains('county')]
# show the uniqe ending words in the state_county column (after the last space)
county_level_data['state_county'].str.split('_').str[-1].unique()

In [ ]:
# show all rows where state_county starts with maine
county_level_data[county_level_data['state_county'].str.startswith('maine')]

In [ ]:
# show all rows where state_county starts with maine
county_level_data[county_level_data['state_county'].str.startswith('maine')]

In [ ]:
# scatterplot of PCTPOVALL_2021 vs Unemployment_rate_2021, include a best fit line
fig, ax = plt.subplots()
merged_geo.plot.scatter(x='PCTPOVALL_2021', y='Unemployment_rate_2021', ax=ax)
plt.show()

In [53]:
# read in the 2016 and 2020 election data
election_2016 = pd.read_csv('data/election/2016_US_County_Level_Presidential_Results.csv')
election_2020 = pd.read_csv('data/election/2020_US_County_Level_Presidential_Results.csv')

In [86]:
# read the 2012 election data
election_2012 = pd.read_csv('data/election/US_County_Level_Presidential_Results_12-16.csv')

In [87]:
election_2012

,Unnamed: 0,combined_fips,votes_dem_2016,votes_gop_2016,total_votes_2016,per_dem_2016,per_gop_2016,diff_2016,per_point_diff_2016,state_abbr,county_name,FIPS,total_votes_2012,votes_dem_2012,votes_gop_2012,county_fips,state_fips,per_dem_2012,per_gop_2012,diff_2012,per_point_diff_2012
0,0,2013,93003.0,130413.0,246588.0,0.377159,0.528870,37410,-0.151711,AK,Alaska,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2016,93003.0,130413.0,246588.0,0.377159,0.528870,37410,-0.151711,AK,Alaska,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2020,93003.0,130413.0,246588.0,0.377159,0.528870,37410,-0.151711,AK,Alaska,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2050,93003.0,130413.0,246588.0,0.377159,0.528870,37410,-0.151711,AK,Alaska,2050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2060,93003.0,130413.0,246588.0,0.377159,0.528870,37410,-0.151711,AK,Alaska,2060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,3136,56037,3233.0,12153.0,16661.0,0.194046,0.729428,8920,-0.535382,WY,Sweetwater County,56037,16750.0,4773.0,11427.0,37.0,56.0,0.284955,0.682209,6654.0,-0.397254
3137,3137,56039,7313.0,3920.0,12176.0,0.600608,0.321945,3393,0.278663,WY,Teton County,56039,11356.0,6211.0,4858.0,39.0,56.0,0.546936,0.427791,1353.0,0.119144
3138,3138,56041,1202.0,6154.0,8053.0,0.149261,0.764187,4952,-0.614926,WY,Uinta County,56041,8453.0,1628.0,6613.0,41.0,56.0,0.192594,0.782326,4985.0,-0.589731
3139,3139,56043,532.0,2911.0,3715.0,0.143203,0.783580,2379,-0.640377,WY,Washakie County,56043,3911.0,794.0,3013.0,43.0,56.0,0.203017,0.770391,2219.0,-0.567374


In [89]:
# drop columns from election_2012 containing any of the following strings in the col name: '2016'
cols_to_drop = [col for col in election_2012.columns if '2016' in col]
election_2012.drop(columns=cols_to_drop, inplace=True)

In [59]:
election_2020

,state_name,county_fips,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff
0,Alabama,1001,Autauga County,19838,7503,27770,12335,0.714368,0.270184,0.444184
1,Alabama,1003,Baldwin County,83544,24578,109679,58966,0.761714,0.224090,0.537623
2,Alabama,1005,Barbour County,5622,4816,10518,806,0.534512,0.457882,0.076631
3,Alabama,1007,Bibb County,7525,1986,9595,5539,0.784263,0.206983,0.577280
4,Alabama,1009,Blount County,24711,2640,27588,22071,0.895716,0.095694,0.800022
...,...,...,...,...,...,...,...,...,...,...
3147,Wyoming,56037,Sweetwater County,12229,3823,16603,8406,0.736554,0.230260,0.506294
3148,Wyoming,56039,Teton County,4341,9848,14677,-5507,0.295769,0.670982,-0.375213
3149,Wyoming,56041,Uinta County,7496,1591,9402,5905,0.797277,0.169219,0.628058
3150,Wyoming,56043,Washakie County,3245,651,4012,2594,0.808824,0.162263,0.646560


In [91]:
election_2016 = election_2016[~election_2016.state_abbr.isin(['AK', 'HI', 'DC'])]
election_2020 = election_2020[~election_2020.state_name.isin(['Alaska', 'Hawaii', 'District of Columbia'])]
election_2012 = election_2012[~election_2012.state_abbr.isin(['AK', 'HI', 'DC'])]

In [92]:
election_2012

,Unnamed: 0,combined_fips,state_abbr,county_name,FIPS,total_votes_2012,votes_dem_2012,votes_gop_2012,county_fips,state_fips,per_dem_2012,per_gop_2012,diff_2012,per_point_diff_2012
29,29,1001,AL,Autauga County,1001,23909.0,6354.0,17366.0,1.0,1.0,0.265758,0.726337,11012.0,-0.460580
30,30,1003,AL,Baldwin County,1003,84988.0,18329.0,65772.0,3.0,1.0,0.215666,0.773897,47443.0,-0.558232
31,31,1005,AL,Barbour County,1005,11459.0,5873.0,5539.0,5.0,1.0,0.512523,0.483376,334.0,0.029147
32,32,1007,AL,Bibb County,1007,8391.0,2200.0,6131.0,7.0,1.0,0.262186,0.730664,3931.0,-0.468478
33,33,1009,AL,Blount County,1009,23980.0,2961.0,20741.0,9.0,1.0,0.123478,0.864929,17780.0,-0.741451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,3136,56037,WY,Sweetwater County,56037,16750.0,4773.0,11427.0,37.0,56.0,0.284955,0.682209,6654.0,-0.397254
3137,3137,56039,WY,Teton County,56039,11356.0,6211.0,4858.0,39.0,56.0,0.546936,0.427791,1353.0,0.119144
3138,3138,56041,WY,Uinta County,56041,8453.0,1628.0,6613.0,41.0,56.0,0.192594,0.782326,4985.0,-0.589731
3139,3139,56043,WY,Washakie County,56043,3911.0,794.0,3013.0,43.0,56.0,0.203017,0.770391,2219.0,-0.567374


In [65]:
election_2016

,Unnamed: 0,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips
29,29,5908.0,18110.0,24661.0,0.239569,0.734358,"12,202",49.48%,AL,Autauga County,1001
30,30,18409.0,72780.0,94090.0,0.195653,0.773515,"54,371",57.79%,AL,Baldwin County,1003
31,31,4848.0,5431.0,10390.0,0.466603,0.522714,583,5.61%,AL,Barbour County,1005
32,32,1874.0,6733.0,8748.0,0.214220,0.769662,"4,859",55.54%,AL,Bibb County,1007
33,33,2150.0,22808.0,25384.0,0.084699,0.898519,"20,658",81.38%,AL,Blount County,1009
...,...,...,...,...,...,...,...,...,...,...,...
3136,3136,3233.0,12153.0,16661.0,0.194046,0.729428,"8,920",53.54%,WY,Sweetwater County,56037
3137,3137,7313.0,3920.0,12176.0,0.600608,0.321945,"3,393",27.87%,WY,Teton County,56039
3138,3138,1202.0,6154.0,8053.0,0.149261,0.764187,"4,952",61.49%,WY,Uinta County,56041
3139,3139,532.0,2911.0,3715.0,0.143203,0.783580,"2,379",64.04%,WY,Washakie County,56043


In [64]:
election_2020

,state_name,county_fips,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff
0,Alabama,1001,Autauga County,19838,7503,27770,12335,0.714368,0.270184,0.444184
1,Alabama,1003,Baldwin County,83544,24578,109679,58966,0.761714,0.224090,0.537623
2,Alabama,1005,Barbour County,5622,4816,10518,806,0.534512,0.457882,0.076631
3,Alabama,1007,Bibb County,7525,1986,9595,5539,0.784263,0.206983,0.577280
4,Alabama,1009,Blount County,24711,2640,27588,22071,0.895716,0.095694,0.800022
...,...,...,...,...,...,...,...,...,...,...
3147,Wyoming,56037,Sweetwater County,12229,3823,16603,8406,0.736554,0.230260,0.506294
3148,Wyoming,56039,Teton County,4341,9848,14677,-5507,0.295769,0.670982,-0.375213
3149,Wyoming,56041,Uinta County,7496,1591,9402,5905,0.797277,0.169219,0.628058
3150,Wyoming,56043,Washakie County,3245,651,4012,2594,0.808824,0.162263,0.646560


In [74]:
# in both election_2016 and election_2020, append _2016 or _2020 to the end of the column names where the name is in votes_gop	votes_dem	total_votes	diff	per_gop	per_dem
election_2016.columns = election_2016.columns.str.replace('votes_gop', 'votes_gop_2016')
election_2016.columns = election_2016.columns.str.replace('votes_dem', 'votes_dem_2016')
election_2016.columns = election_2016.columns.str.replace('total_votes', 'total_votes_2016')
election_2016.columns = election_2016.columns.str.replace('diff', 'diff_2016')
election_2016.columns = election_2016.columns.str.replace('per_gop', 'per_gop_2016')
election_2016.columns = election_2016.columns.str.replace('per_dem', 'per_dem_2016')

election_2020.columns = election_2020.columns.str.replace('votes_gop', 'votes_gop_2020')
election_2020.columns = election_2020.columns.str.replace('votes_dem', 'votes_dem_2020')
election_2020.columns = election_2020.columns.str.replace('total_votes', 'total_votes_2020')
election_2020.columns = election_2020.columns.str.replace('diff', 'diff_2020')
election_2020.columns = election_2020.columns.str.replace('per_gop', 'per_gop_2020')
election_2020.columns = election_2020.columns.str.replace('per_dem', 'per_dem_2020')

In [66]:
# rename county_fips in election_2020 to combined_fips
election_2020.rename(columns={'county_fips': 'combined_fips'}, inplace=True)

C:\Users\bovam\AppData\Local\Temp\ipykernel_26600\2550907012.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  election_2020.rename(columns={'county_fips': 'combined_fips'}, inplace=True)


In [93]:
# change the combined_fips column in both election_2016 and election_2020 to string
election_2016['combined_fips'] = election_2016['combined_fips'].astype(str)
election_2020['combined_fips'] = election_2020['combined_fips'].astype(str)
election_2012['combined_fips'] = election_2012['combined_fips'].astype(str)

C:\Users\bovam\AppData\Local\Temp\ipykernel_26600\2193909652.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  election_2012['combined_fips'] = election_2012['combined_fips'].astype(str)


In [106]:
# merge election_2020 with merged_geo on the FIPS Code column
election_2012_merged = election_2012.merge(merged_geo, left_on='combined_fips', right_on='FIPS Code')

In [107]:
election_2012_merged

,Unnamed: 0,combined_fips,state_abbr,county_name,FIPS,total_votes_2012,votes_dem_2012,votes_gop_2012,county_fips,state_fips,per_dem_2012,per_gop_2012,diff_2012,per_point_diff_2012,FIPS Code,State,Area name,"Less than a high school diploma, 2000","High school diploma only, 2000","Some college or associate's degree, 2000","Bachelor's degree or higher, 2000","Percent of adults with less than a high school diploma, 2000","Percent of adults with a high school diploma only, 2000","Percent of adults completing some college or associate's degree, 2000","Percent of adults with a bachelor's degree or higher, 2000","Less than a high school diploma, 2008-12","High school diploma only, 2008-12","Some college or associate's degree, 2008-12","Bachelor's degree or higher, 2008-12","Percent of adults with less than a high school diploma, 2008-12","Percent of adults with a high school diploma only, 2008-12","Percent of adults completing some college or associate's degree, 2008-12","Percent of adults with a bachelor's degree or higher, 2008-12","Less than a high school diploma, 2018-22","High school diploma only, 2018-22","Some college or associate's degree, 2018-22","Bachelor's degree or higher, 2018-22","Percent of adults with less than a high school diploma, 2018-22","Percent of adults with a high school diploma only, 2018-22","Percent of adults completing some college or associate's degree, 2018-22","Percent of adults with a bachelor's degree or higher, 2018-22",ALAND,AWATER,geometry,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,Civilian_labor_force_2001,Employed_2001,Unemployed_2001,Unemployment_rate_2001,Civilian_labor_force_2002,Employed_2002,Unemployed_2002,Unemployment_rate_2002,Civilian_labor_force_2003,Employed_2003,Unemployed_2003,Unemployment_rate_2003,Civilian_labor_force_2004,Employed_2004,Unemployed_2004,Unemployment_rate_2004,Civilian_labor_force_2005,Employed_2005,Unemployed_2005,Unemployment_rate_2005,Civilian_labor_force_2006,Employed_2006,Unemployed_2006,Unemployment_rate_2006,Civilian_labor_force_2007,Employed_2007,Unemployed_2007,Unemployment_rate_2007,Civilian_labor_force_2008,Employed_2008,Unemployed_2008,Unemployment_rate_2008,Civilian_labor_force_2009,Employed_2009,Unemployed_2009,Unemployment_rate_2009,Civilian_labor_force_2010,Employed_2010,Unemployed_2010,Unemployment_rate_2010,Civilian_labor_force_2011,Employed_2011,Unemployed_2011,Unemployment_rate_2011,Civilian_labor_force_2012,Employed_2012,Unemployed_2012,Unemployment_rate_2012,Civilian_labor_force_2013,Employed_2013,Unemployed_2013,Unemployment_rate_2013,Civilian_labor_force_2014,Employed_2014,Unemployed_2014,Unemployment_rate_2014,Civilian_labor_force_2015,Employed_2015,Unemployed_2015,Unemployment_rate_2015,Civilian_labor_force_2016,Employed_2016,Unemployed_2016,Unemployment_rate_2016,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Civilian_labor_force_2021,Employed_2021,Unemployed_2021,Unemployment_rate_2021,Civilian_labor_force_2022,Employed_2022,Unemployed_2022,Unemployment_rate_2022,Median_Household_Income_2021,Med_HH_Income_Percent_of_State_Total_2021,POVALL_2021,CI90LBALL_2021,CI90UBALL_2021,PCTPOVALL_2021,CI90LBALLP_2021,CI90UBALLP_2021,POV017_2021,CI90LB017_2021,CI90UB017_2021,PCTPOV017_2021,CI90LB017P_2021,CI90UB017P_2021,POV517_2021,CI90LB517_2021,CI90UB517_2021,PCTPOV517_2021,CI90LB517P_2021,CI90UB517P_2021,MEDHHINC_2021,CI90LBINC_2021,CI90UBINC_2021,POV04_2021,CI90LB04_2021,CI90UB04_2021,PCTPOV04_2021,CI90LB04P_2021,CI90UB04P_2021,FIPStxt,Economic_typology_2015,CENSUS_2020_POP,ESTIMATES_BASE_2020,POP_ESTIMATE_2020,POP_ESTIMATE_2021,POP_ESTIMATE_2022,POP_ESTIMATE_2023,N_POP_CHG_2020,N_POP_CHG_2021,N_POP_CHG_2022,N_POP_CHG_2023,BIRTHS_2020,BI

In [108]:
# merge 2016 into the 2020_merged data on the combined_fips column
election_2012_2016_merged = election_2012_merged.merge(election_2016, on='combined_fips')

In [110]:
election_2012_2016_merged

,Unnamed: 0_x,combined_fips,state_abbr_x,county_name_x,FIPS,total_votes_2012,votes_dem_2012,votes_gop_2012,county_fips,state_fips,per_dem_2012,per_gop_2012,diff_2012,per_point_diff_2012,FIPS Code,State,Area name,"Less than a high school diploma, 2000","High school diploma only, 2000","Some college or associate's degree, 2000","Bachelor's degree or higher, 2000","Percent of adults with less than a high school diploma, 2000","Percent of adults with a high school diploma only, 2000","Percent of adults completing some college or associate's degree, 2000","Percent of adults with a bachelor's degree or higher, 2000","Less than a high school diploma, 2008-12","High school diploma only, 2008-12","Some college or associate's degree, 2008-12","Bachelor's degree or higher, 2008-12","Percent of adults with less than a high school diploma, 2008-12","Percent of adults with a high school diploma only, 2008-12","Percent of adults completing some college or associate's degree, 2008-12","Percent of adults with a bachelor's degree or higher, 2008-12","Less than a high school diploma, 2018-22","High school diploma only, 2018-22","Some college or associate's degree, 2018-22","Bachelor's degree or higher, 2018-22","Percent of adults with less than a high school diploma, 2018-22","Percent of adults with a high school diploma only, 2018-22","Percent of adults completing some college or associate's degree, 2018-22","Percent of adults with a bachelor's degree or higher, 2018-22",ALAND,AWATER,geometry,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,Civilian_labor_force_2001,Employed_2001,Unemployed_2001,Unemployment_rate_2001,Civilian_labor_force_2002,Employed_2002,Unemployed_2002,Unemployment_rate_2002,Civilian_labor_force_2003,Employed_2003,Unemployed_2003,Unemployment_rate_2003,Civilian_labor_force_2004,Employed_2004,Unemployed_2004,Unemployment_rate_2004,Civilian_labor_force_2005,Employed_2005,Unemployed_2005,Unemployment_rate_2005,Civilian_labor_force_2006,Employed_2006,Unemployed_2006,Unemployment_rate_2006,Civilian_labor_force_2007,Employed_2007,Unemployed_2007,Unemployment_rate_2007,Civilian_labor_force_2008,Employed_2008,Unemployed_2008,Unemployment_rate_2008,Civilian_labor_force_2009,Employed_2009,Unemployed_2009,Unemployment_rate_2009,Civilian_labor_force_2010,Employed_2010,Unemployed_2010,Unemployment_rate_2010,Civilian_labor_force_2011,Employed_2011,Unemployed_2011,Unemployment_rate_2011,Civilian_labor_force_2012,Employed_2012,Unemployed_2012,Unemployment_rate_2012,Civilian_labor_force_2013,Employed_2013,Unemployed_2013,Unemployment_rate_2013,Civilian_labor_force_2014,Employed_2014,Unemployed_2014,Unemployment_rate_2014,Civilian_labor_force_2015,Employed_2015,Unemployed_2015,Unemployment_rate_2015,Civilian_labor_force_2016,Employed_2016,Unemployed_2016,Unemployment_rate_2016,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Civilian_labor_force_2021,Employed_2021,Unemployed_2021,Unemployment_rate_2021,Civilian_labor_force_2022,Employed_2022,Unemployed_2022,Unemployment_rate_2022,Median_Household_Income_2021,Med_HH_Income_Percent_of_State_Total_2021,POVALL_2021,CI90LBALL_2021,CI90UBALL_2021,PCTPOVALL_2021,CI90LBALLP_2021,CI90UBALLP_2021,POV017_2021,CI90LB017_2021,CI90UB017_2021,PCTPOV017_2021,CI90LB017P_2021,CI90UB017P_2021,POV517_2021,CI90LB517_2021,CI90UB517_2021,PCTPOV517_2021,CI90LB517P_2021,CI90UB517P_2021,MEDHHINC_2021,CI90LBINC_2021,CI90UBINC_2021,POV04_2021,CI90LB04_2021,CI90UB04_2021,PCTPOV04_2021,CI90LB04P_2021,CI90UB04P_2021,FIPStxt,Economic_typology_2015,CENSUS_2020_POP,ESTIMATES_BASE_2020,POP_ESTIMATE_2020,POP_ESTIMATE_2021,POP_ESTIMATE_2022,POP_ESTIMATE_2023,N_POP_CHG_2020,N_POP_CHG_2021,N_POP_CHG_2022,N_POP_CHG_2023,BIRTHS_2

In [111]:
# merge 2012 into the 2016_2020_merged data on the combined_fips column
election_2012_2016_2020_merged = election_2012_2016_merged.merge(election_2020, on='combined_fips')

In [112]:
election_2012_2016_2020_merged

,Unnamed: 0_x,combined_fips,state_abbr_x,county_name_x,FIPS,total_votes_2012,votes_dem_2012,votes_gop_2012,county_fips,state_fips,per_dem_2012,per_gop_2012,diff_2012,per_point_diff_2012,FIPS Code,State,Area name,"Less than a high school diploma, 2000","High school diploma only, 2000","Some college or associate's degree, 2000","Bachelor's degree or higher, 2000","Percent of adults with less than a high school diploma, 2000","Percent of adults with a high school diploma only, 2000","Percent of adults completing some college or associate's degree, 2000","Percent of adults with a bachelor's degree or higher, 2000","Less than a high school diploma, 2008-12","High school diploma only, 2008-12","Some college or associate's degree, 2008-12","Bachelor's degree or higher, 2008-12","Percent of adults with less than a high school diploma, 2008-12","Percent of adults with a high school diploma only, 2008-12","Percent of adults completing some college or associate's degree, 2008-12","Percent of adults with a bachelor's degree or higher, 2008-12","Less than a high school diploma, 2018-22","High school diploma only, 2018-22","Some college or associate's degree, 2018-22","Bachelor's degree or higher, 2018-22","Percent of adults with less than a high school diploma, 2018-22","Percent of adults with a high school diploma only, 2018-22","Percent of adults completing some college or associate's degree, 2018-22","Percent of adults with a bachelor's degree or higher, 2018-22",ALAND,AWATER,geometry,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,Civilian_labor_force_2001,Employed_2001,Unemployed_2001,Unemployment_rate_2001,Civilian_labor_force_2002,Employed_2002,Unemployed_2002,Unemployment_rate_2002,Civilian_labor_force_2003,Employed_2003,Unemployed_2003,Unemployment_rate_2003,Civilian_labor_force_2004,Employed_2004,Unemployed_2004,Unemployment_rate_2004,Civilian_labor_force_2005,Employed_2005,Unemployed_2005,Unemployment_rate_2005,Civilian_labor_force_2006,Employed_2006,Unemployed_2006,Unemployment_rate_2006,Civilian_labor_force_2007,Employed_2007,Unemployed_2007,Unemployment_rate_2007,Civilian_labor_force_2008,Employed_2008,Unemployed_2008,Unemployment_rate_2008,Civilian_labor_force_2009,Employed_2009,Unemployed_2009,Unemployment_rate_2009,Civilian_labor_force_2010,Employed_2010,Unemployed_2010,Unemployment_rate_2010,Civilian_labor_force_2011,Employed_2011,Unemployed_2011,Unemployment_rate_2011,Civilian_labor_force_2012,Employed_2012,Unemployed_2012,Unemployment_rate_2012,Civilian_labor_force_2013,Employed_2013,Unemployed_2013,Unemployment_rate_2013,Civilian_labor_force_2014,Employed_2014,Unemployed_2014,Unemployment_rate_2014,Civilian_labor_force_2015,Employed_2015,Unemployed_2015,Unemployment_rate_2015,Civilian_labor_force_2016,Employed_2016,Unemployed_2016,Unemployment_rate_2016,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Civilian_labor_force_2021,Employed_2021,Unemployed_2021,Unemployment_rate_2021,Civilian_labor_force_2022,Employed_2022,Unemployed_2022,Unemployment_rate_2022,Median_Household_Income_2021,Med_HH_Income_Percent_of_State_Total_2021,POVALL_2021,CI90LBALL_2021,CI90UBALL_2021,PCTPOVALL_2021,CI90LBALLP_2021,CI90UBALLP_2021,POV017_2021,CI90LB017_2021,CI90UB017_2021,PCTPOV017_2021,CI90LB017P_2021,CI90UB017P_2021,POV517_2021,CI90LB517_2021,CI90UB517_2021,PCTPOV517_2021,CI90LB517P_2021,CI90UB517P_2021,MEDHHINC_2021,CI90LBINC_2021,CI90UBINC_2021,POV04_2021,CI90LB04_2021,CI90UB04_2021,PCTPOV04_2021,CI90LB04P_2021,CI90UB04P_2021,FIPStxt,Economic_typology_2015,CENSUS_2020_POP,ESTIMATES_BASE_2020,POP_ESTIMATE_2020,POP_ESTIMATE_2021,POP_ESTIMATE_2022,POP_ESTIMATE_2023,N_POP_CHG_2020,N_POP_CHG_2021,N_POP_CHG_2022,N_POP_CHG_2023,BIRTHS_2

In [113]:
# remove all commas from values in all columns with object type
election_2012_2016_2020_merged = election_2012_2016_2020_merged.apply(lambda x: x.str.replace(',', '') if x.dtype == 'object' else x)

In [114]:
election_2012_2016_2020_merged.drop(columns=['POV04_2021', 'CI90LB04_2021', 'CI90UB04_2021', 'PCTPOV04_2021', 'CI90LB04P_2021', 'CI90UB04P_2021'], inplace=True)

In [116]:
election_2012_2016_2020_merged.drop(columns=['county_name_x', 'combined_fips', 'FIPStxt', 'county_name_y'], inplace=True)

In [118]:
election_2012_2016_2020_merged.drop(columns=['Unnamed: 0_x', 'FIPS', 'county_fips', 'state_fips', 'state_abbr_y', 'Unnamed: 0_y', 'Area name'], inplace=True)

In [120]:
election_2012_2016_2020_merged.drop(columns=['State'], inplace=True)

In [105]:
# election_2012_2016_2020_merged, show columns that contain 2012
election_2012_2016_2020_merged.columns[election_2012_2016_2020_merged.columns.str.contains('2012')]

Index(['Civilian_labor_force_2012', 'Employed_2012', 'Unemployed_2012',
       'Unemployment_rate_2012'],
      dtype='object')

In [121]:
# in election_2012_2016_2020_merged, create a columns called winner_2012, winner_2016 and winner_2020 that is the name of the winner of the election in that year
election_2012_2016_2020_merged['winner_2012'] = np.where(election_2012_2016_2020_merged['per_gop_2012'] > election_2012_2016_2020_merged['per_dem_2012'], 'GOP', 'DEM')
election_2012_2016_2020_merged['winner_2016'] = np.where(election_2012_2016_2020_merged['per_gop_2016'] > election_2012_2016_2020_merged['per_dem_2016'], 'GOP', 'DEM')
election_2012_2016_2020_merged['winner_2020'] = np.where(election_2012_2016_2020_merged['per_gop_2020'] > election_2012_2016_2020_merged['per_dem_2020'], 'GOP', 'DEM')

In [123]:
# rename state_abbr_x to state_abbr
election_2012_2016_2020_merged.rename(columns={'state_abbr_x': 'state_abbr'}, inplace=True)

In [52]:
# president counties 2020
president_counties_2020 = pd.read_csv('data/election/president_county_candidate.csv')

In [ ]:
president_counties_2020 = president_counties_2020[~president_counties_2020.state.isin(['Alaska', 'Hawaii', 'District of Columbia'])]
# drop rows where party is not 'DEM' or 'REP'
president_counties_2020 = president_counties_2020[president_counties_2020.party.isin(['DEM', 'REP'])]

In [ ]:
len(president_counties_2020.state.unique())

In [ ]:
president_counties_2020['state_county'] = president_counties_2020['state'] + '_' + president_counties_2020['county']

In [ ]:
# we now reduce the number of rows by having each county appear only once.  the columns candidate, party, total_votes, won are translated into columns as such: candidate_dem, candidate_rep, party_dem, party_rep, total_votes_dem, total_votes_rep, won_dem, won_rep
president_counties_2020_fixed = president_counties_2020.pivot(index='state_county', columns='party', values=['candidate', 'total_votes', 'won'])
president_counties_2020_fixed.columns = ['_'.join(col).strip() for col in president_counties_2020_fixed.columns.values]
president_counties_2020_fixed.reset_index(inplace=True)
president_counties_2020_fixed

In [ ]:
# add a column to president_counties_2020_fixed that is called winner_2020, which is 'DEM' if won_dem == True, 'REP' if won_rep == True
president_counties_2020_fixed['winner_2020'] = np.where(president_counties_2020_fixed['won_DEM'] == True, 'DEM', 'REP')
president_counties_2020_fixed

In [ ]:
# show rows in state_county that do not have the word 'County' in them
president_counties_2020_fixed[president_counties_2020_fixed.state_county.str.contains('city')]

In [ ]:
# for values in predident_counties_2020_fixed where state_county does not contain 'Parish' or 'city', append ' County' to the end of the value
president_counties_2020_fixed['state_county'] = np.where(president_counties_2020_fixed['state_county'].str.contains('Parish') | president_counties_2020_fixed['state_county'].str.contains('city') | president_counties_2020_fixed['state_county'].str.contains('County'), president_counties_2020_fixed['state_county'], president_counties_2020_fixed['state_county'] + ' County')

In [ ]:
president_counties_2020_fixed

In [ ]:
def diagnose_merge_issues(df1, df2, key_column, df1_name='Left DataFrame', df2_name='Right DataFrame'):
    """
    Diagnose why certain rows are being dropped in a merge operation.
    
    Args:
        df1 (pandas.DataFrame): First DataFrame
        df2 (pandas.DataFrame): Second DataFrame
        key_column (str): Column name used for merging
        df1_name (str): Name for first DataFrame for reporting
        df2_name (str): Name for second DataFrame for reporting
    """
    # Check for duplicates in merge column
    print(f"\nChecking for duplicates in merge column '{key_column}':")
    print(f"{df1_name} duplicates: {df1[key_column].duplicated().sum()}")
    print(f"{df2_name} duplicates: {df2[key_column].duplicated().sum()}")
    
    if df1[key_column].duplicated().sum() > 0:
        print(f"\nDuplicate values in {df1_name}:")
        print(df1[df1[key_column].duplicated(keep=False)][key_column].sort_values())
    
    if df2[key_column].duplicated().sum() > 0:
        print(f"\nDuplicate values in {df2_name}:")
        print(df2[df2[key_column].duplicated(keep=False)][key_column].sort_values())
    
    # Check for missing values
    print(f"\nChecking for missing values in '{key_column}':")
    print(f"{df1_name} missing values: {df1[key_column].isna().sum()}")
    print(f"{df2_name} missing values: {df2[key_column].isna().sum()}")
    
    # Compare value distributions
    print(f"\nComparing unique values in '{key_column}':")
    df1_values = set(df1[key_column].dropna())
    df2_values = set(df2[key_column].dropna())
    
    print(f"\nTotal unique values:")
    print(f"{df1_name}: {len(df1_values)}")
    print(f"{df2_name}: {len(df2_values)}")
    
    # Find values in df1 that aren't in df2 and vice versa
    only_in_df1 = df1_values - df2_values
    only_in_df2 = df2_values - df1_values
    
    if only_in_df1:
        print(f"\nValues only in {df1_name} (showing first 10):")
        print(sorted(list(only_in_df1))[:10])
    
    if only_in_df2:
        print(f"\nValues only in {df2_name} (showing first 10):")
        print(sorted(list(only_in_df2))[:10])
    
    # Check for leading/trailing whitespace
    print(f"\nChecking for leading/trailing whitespace in '{key_column}':")
    df1_space = df1[df1[key_column].astype(str).str.strip() != df1[key_column].astype(str)]
    df2_space = df2[df2[key_column].astype(str).str.strip() != df2[key_column].astype(str)]
    
    if len(df1_space) > 0:
        print(f"\nValues with whitespace in {df1_name} (showing first 5):")
        print(df1_space[key_column].head())
    
    if len(df2_space) > 0:
        print(f"\nValues with whitespace in {df2_name} (showing first 5):")
        print(df2_space[key_column].head())
    
    # Check for case differences
    print(f"\nChecking for case differences:")
    df1_lower = set(df1[key_column].astype(str).str.lower())
    df2_lower = set(df2[key_column].astype(str).str.lower())
    case_diff = len(df1_values.intersection(df2_values)) != len(df1_lower.intersection(df2_lower))
    
    if case_diff:
        print("Warning: Found case differences in matching values!")
        
    # Sample a few unmatched values for detailed comparison
    if only_in_df1 or only_in_df2:
        print("\nDetailed comparison of a few unmatched values:")
        for val in list(only_in_df1)[:3]:
            close_matches = [x for x in df2_values if str(val).lower() in str(x).lower() or str(x).lower() in str(val).lower()]
            if close_matches:
                print(f"\nValue in {df1_name}: '{val}'")
                print(f"Similar values in {df2_name}: {close_matches}")

def fix_merge_issues(df1, df2, key_column):
    """
    Try to fix common merge issues in both DataFrames.
    
    Args:
        df1 (pandas.DataFrame): First DataFrame
        df2 (pandas.DataFrame): Second DataFrame
        key_column (str): Column name used for merging
        
    Returns:
        tuple: (cleaned_df1, cleaned_df2)
    """
    # Create copies to avoid modifying originals
    df1_clean = df1.copy()
    df2_clean = df2.copy()
    
    # Strip whitespace
    if df1_clean[key_column].dtype == object:
        df1_clean[key_column] = df1_clean[key_column].str.strip()
    if df2_clean[key_column].dtype == object:
        df2_clean[key_column] = df2_clean[key_column].str.strip()
    
    # Remove any duplicate rows based on the key column
    df1_clean = df1_clean.drop_duplicates(subset=[key_column])
    df2_clean = df2_clean.drop_duplicates(subset=[key_column])
    
    return df1_clean, df2_clean

In [ ]:
diagnose_merge_issues(county_level_data, president_counties_2020_fixed, 'state_county', 'county_level_data', 'president_counties_2020_fixed')

In [ ]:
# change state_county to lower case for president_counties_2020_fixed
president_counties_2020_fixed['state_county'] = president_counties_2020_fixed['state_county'].str.lower()

In [ ]:
# merge the president_counties_2020_fixed data with the county_level_data data on the state_county column
president_counties_2020_merged = county_level_data.merge(president_counties_2020_fixed, on='state_county')

In [ ]:
len(president_counties_2020_merged.State.unique())

In [ ]:
president_counties_2020_merged.to_file('data/election/final_data/county_data_with_2020_election.geojson', driver='GeoJSON')

In [ ]:
president_counties_2020_merged = gpd.read_file('data/election/final_data/county_data_with_2020_election.geojson')

In [ ]:
# remove all commas from values in all columns with object type
president_counties_2020_merged = president_counties_2020_merged.apply(lambda x: x.str.replace(',', '') if x.dtype == 'object' else x)

In [ ]:
president_counties_2020_merged.drop(columns=['POV04_2021', 'CI90LB04_2021', 'CI90UB04_2021', 'PCTPOV04_2021', 'CI90LB04P_2021', 'CI90UB04P_2021'], inplace=True)

In [ ]:
len(president_counties_2020_merged.State.unique())

In [125]:
# make election_2012_2016_2020_merged as gdf
president_counties_merged = gpd.GeoDataFrame(election_2012_2016_2020_merged, geometry=election_2012_2016_2020_merged['geometry'])

In [ ]:
# save president_counties_2020_merged as geojson
president_counties_2020_merged.to_file('data/election/final_data/county_data_final.geojson', driver='GeoJSON')

In [126]:
president_counties_merged.to_file('data/election/final_data/county_data_with_elections_2012_2016_2020.geojson', driver='GeoJSON')

In [ ]:
counties_2020_data = gpd.read_file('data/election/final_data/county_data_with_2020_election.csv')

In [ ]:
counties_2020_data

In [ ]:
counties_2020_data.set_geometry('geometry')

In [ ]:
# drop POV04_2021	CI90LB04_2021	CI90UB04_2021	PCTPOV04_2021	CI90LB04P_2021	CI90UB04P_2021 from counties_2020_data
counties_2020_data.drop(columns=['POV04_2021', 'CI90LB04_2021', 'CI90UB04_2021', 'PCTPOV04_2021', 'CI90LB04P_2021', 'CI90UB04P_2021'], inplace=True)
# save as geojson
counties_2020_data.to_file('data/election/final_data/county_data_with_2020_election_results.geojson', driver='GeoJSON')